In [5]:
import alpaca_trade_api as tradeapi
import config
import logging
from alpaca_trade_api.rest import REST, TimeFrame
import datetime as dt
import pandas as pd
from datetime import date
import datetime
from pytz import timezone

#ally api endpoint creation
api = tradeapi.REST(key_id=config.LIVE_API_KEY, secret_key=config.LIVE_SECRET_KEY, api_version='v2',
                                        base_url=config.LIVE_API_BASE_URL)

source_greeting = str("For the data source: Enter M if you will be running Golden Potato using a list of default \ndaily top moving stocks. Enter L if you will be manually entering a list of stock tickers. \nFinally, enter F if you want to enter the name of a CSV file containing a list of stock tickers. \nPlease refer to the documentation for the preferred format. ")


#create automated default 2 week historical data window:
#create starting and ending dates, set to nearest friday and two weeks from nearest friday
get_first_friday = dt.date.today()
while get_first_friday.weekday() != 4:
    get_first_friday += dt.timedelta(1)
two_weeks = dt.timedelta(days=14)
two_weeks_from_nearest_friday = get_first_friday + two_weeks
two_weeks_from_nearest_friday_formatted = two_weeks_from_nearest_friday.strftime("%Y-%m-%d")



#Collect inputs from user

quick_list = []
option_price = []

def input_collector():
    print(source_greeting)
    source = input("Enter M, L, or F: ")
    print(source)
    date_set = input("For the data range: The default is 2 weeks of historical data, or entering a customized range is possible. Enter D for default or C to customize the date range: ")
    price_set = input("For the price range, Enter D for the default price range of no more than $1 per option premium, or M to manually set a price cutoff: ")
    s_date_list = []
    e_date_list = []
    if source == "L":
        stock_list = input("Enter stock tickers separated by a comma: ")
        quick_list.append(stock_list)
        out_quick = pd.DataFrame(quick_list)
        out_quick.to_csv(r'D:\mainline\goldenpotato\outputs\quick_list.csv')
    if source == "F":
        source = pd.read_csv(r"D:\mainline\goldenpotato\datasets\sample_csv_list.csv")
        quick_list.append(source)
            
    if date_set == "C":
        s_date = input("Enter start date in YYYY-MM-DD format: ")
        for i in s_date:
            temps = s_date
            s_date_list.append(temps)
        e_date = input("Enter end date in similar format: ")
        for i in e_date:
            tempe = e_date
            e_date_list.append(tempe)
    if date_set == "D":
        s_date = get_first_friday
        e_date = two_weeks_from_nearest_friday_formatted
    if price_set == "D":
        optiond = 1
        option_price.append(optiond)
    if price_set == "M":
        optionc = input("Enter premium cutoff price, this will consider both calls and puts: ")
        option_price.append(optionc)
    print(quick_list)
    print(s_date)
    print(e_date)
    print(option_price)
    format = "%Y-%m-%d"

    
    
def is_market_open():
    clock = api.get_clock()
    check_hours = clock.is_open
    if check_hours is False: 
        print("The market is not open at the moment, Cheddar Butler will run once the market opens.")
    if check_hours is True:
        print("The market is open, Cheddar Butler will now start...")
is_market_open()
    


#create dataframe adding EMA columns to date, stock,     





def dataframe_creator():
    target_symbols_long = []
    target_symbols_short = []
    quote_iter = api.get_bars(i, TimeFrame.Day, start = ninedaysago, end = yesterday, limit=80)._raw
    quotes_df0 = pd.DataFrame(quote_iter)
    quotes_df = pd.DataFrame(index=None)
    quotes_df['Date'] = pd.to_datetime(quotes_df0['t'])
    quotes_df['Date'] = quotes_df['Date'].dt.strftime('%m/%d/%Y')
    quotes_df['Symbol'] = i
    quotes_df['Close'] = quotes_df0['c']
    quotes_df["EMA9"] = quotes_df0['c'].ewm(span=9).mean().round(2)
    quotes_df["EMA20"] = quotes_df0['c'].ewm(span=20).mean().round(2)
    quotes_df["EMA50"] =quotes_df0['c'].ewm(span=50).mean().round(2)
    quotes_df['Volume'] = quotes_df0['v'].apply(lambda x : "{:,}".format(x))
    bool_long = quotes_df["EMA9"].iloc[-1] < quotes_df["Close"].iloc[-1]  ##create Boolean to filter out stocks to go long on.
    if bool_long == True:
        target_list_long = i
        target_symbols_long.append(target_list_long)
        print("Stock ", i, "is within buying parameters. Dataframe: " , "\n")
        print(quotes_df.tail(1))
    else:
        print("Stock", i, "is not within buying parameters")

    bool_short = quotes_df["EMA9"].iloc[-1] > quotes_df["Close"].iloc[-1]  ##create second boolean to filter out stocks to short.
    if bool_short == True:
        target_list_short = i
        target_symbols_short.append(target_list_short)
        print("Stock ", i, "is within selling parameters. Dataframe: " , "\n")
        print(quotes_df.tail(1))
    else:
        print("Stock", i, "is not within selling parameters")


  









excel_file = pd.read_csv(r"D:\mainline\goldenpotato\datasets\sample_csv_list.csv")

now = date.today()
today = now.strftime('%Y-%m-%d')
yesterday = (now - pd.Timedelta('1day')).strftime('%Y-%m-%d')
ninedaysago = (now - pd.Timedelta('9day')).strftime('%Y-%m-%d')


Symbols = excel_file['symbol'].tolist()


    
#call functions

#input_collector()

    

The market is open, Cheddar Butler will now start...


# 